In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('D:\\Dropbox\\stempy')
sys.path.append('D:\\Dropbox\\stemplot')
    
from stemplot import *

from stempy.io import *
from stempy.clustering import *
from stempy.plot import *
from stempy.denoise import *
from stempy.utils import *
from stempy.feature import *
from stempy.spatial import *
from stempy.manifold import *

## Find points from HAADF

In [2]:
dp = r'D:\Dropbox\data\Leyi\Dopant Valency Project\CoWSe2_total 9 datasets\CoWSe2_20211124'+'\\'

num = 18

file_name1 = dp + 'ADF1_{}.dm4'.format(num)

# load image
img = load_image(file_name1)

img = normalize_image(img, 0, 1)

In [3]:
img1 = remove_bg(img, 11)

imshow(img1)

In [4]:
get_window_size(img)

13

## denoising

In [5]:
n_components = 32
patch_size = 32
extraction_step = 4
imgf = denoise_svd(img1, n_components, patch_size, extraction_step) 

Extracting reference patches...
done in 0.27s.
Singular value decomposition...
done in 2.12s.
Reconstructing...
done in 1.20s.


In [6]:
imshow(imgf)

In [7]:
threshold = 0.3
pts = local_max(imgf, min_distance=1, threshold=threshold)

## Clustering using MAADF

In [8]:
file_name2 = dp + 'ADF2_{}.dm4'.format(num)

# load image
img = load_image(file_name2)

img = normalize_image(img, 0, 1)

img2 = remove_bg(img, 11)

In [9]:
imshow(img2)

In [10]:
size = 26
kp = KeyPoints(pts, img2, size)
ps = kp.extract_patches(size) 

In [11]:
imshow(ps)

In [12]:
zps = ZPs(n_max=10, size=ps.shape[1])
zps.fit(ps)
X = zps.moments
X_rot = X.rotinv() 

In [13]:
plot_pca(X, 2)

In [14]:
fg = ForceGraph8(X = X,
n_neighbors=10,
init_mode = 'pca',
num_iterations = 100,
num_negative_samples=10,
force_params1=(0, 2, 1, 1),
force_params2=(2, 5, 5, 1),
divide=0.5)

from time import time
t0 = time()
xy = fg.fit_transform(X)

Construct graph from data...
Initialize 2-d embedding using PCA layout...


In [15]:
l= interactive_clusters(xy, img, kp.pts, ps, clip=False, s=1) 

One cluster has been selected.


In [16]:
len(l.ind), l.ind

(62,
 array([   49,   376,   411,   891,   964,  1129,  1385,  1397,  1499,
         1666,  1713,  1727,  1828,  2152,  2321,  2805,  2850,  2905,
         2946,  3064,  3181,  3481,  3717,  3878,  3981,  4259,  4306,
         4422,  4948,  5115,  5176,  5233,  5323,  5492,  6082,  6288,
         6780,  7094,  7326,  7472,  7475,  7545,  7602,  7700,  7755,
         7867,  8284,  8774,  8828,  9164,  9259,  9693, 10006, 10029,
        10113, 10333, 10566, 10833, 10924, 10978, 11202, 11298],
       dtype=int64))

In [16]:
kp.pts

array([[ 48,   8],
       [ 78,   8],
       [121,   8],
       ...,
       [452, 504],
       [461, 504],
       [482, 504]], dtype=int64)

In [17]:
np.save('pts_Ti_25.npy', kp.pts)

In [18]:
np.save('ind_Ti_25.npy', l.ind)